In [ ]:
using JLD2
using Plots
using Statistics

In [ ]:
file_00 = joinpath(@__DIR__, "EPS_Schmidt_data_0.0.jld2")
file_002 = joinpath(@__DIR__, "EPS_Schmidt_data_0.002.jld2")



d00 = JLD2.load(file_00)
d002 = JLD2.load(file_002)

results_00 = d00["entanglement_spectrum_results"]
results_002 = d002["entanglement_spectrum_results"]


In [ ]:
"""
Plots the Schmidt coefficient tails for a subset of N values on a 2x4 grid.
Compares two datasets (e.g. sigma=0.0 vs sigma=0.002).
"""
function plot_tail_comparison(results_1::Dict, results_2::Dict; 
                              N_subset=[10, 20, 30, 40, 50, 60, 70, 80],
                              label1="σ=0.0", label2="σ=0.002")
    
    # Initialize 2x4 grid
    p = plot(
        layout = (2, 4),
        size = (1200, 600),
        plot_title = "Schmidt Coefficient Tail Behavior (Log Scale)",
        plot_titlefontsize = 16,
        legend = :topright,
        margin = 5Plots.mm
    )

    for (i, N) in enumerate(N_subset)
        if i > 8 break end # Safety check for grid size
        
        # --- Prepare Data ---
        # Sort descending
        v1 = haskey(results_1, N) ? sort(results_1[N], rev=true) : nothing
        v2 = haskey(results_2, N) ? sort(results_2[N], rev=true) : nothing

        # Only add legend labels to the very first subplot
        l1 = (i == 1) ? label1 : ""
        l2 = (i == 1) ? label2 : ""

        # --- Plot Dataset 2 (Orange/0.002) ---
        if v2 !== nothing
            mask = v2 .> 0 # Filter zeros for log plot
            y = v2[mask]
            x = (1:length(v2))[mask]
            
            plot!(p, subplot=i, x, y, 
                seriestype=:scatter, markershape=:circle, markersize=3, 
                markerstrokewidth=0, color=:darkorange, alpha=0.8, label=l2)
            plot!(p, subplot=i, x, y, 
                seriestype=:path, color=:darkorange, alpha=0.5, label="")
        end

        # --- Plot Dataset 1 (Purple/0.0) ---
        if v1 !== nothing
            mask = v1 .> 0
            y = v1[mask]
            x = (1:length(v1))[mask]
            
            plot!(p, subplot=i, x, y, 
                seriestype=:scatter, markershape=:rect, markersize=3, 
                markerstrokewidth=0, color=:purple, alpha=0.4, label=l1)
            plot!(p, subplot=i, x, y, 
                seriestype=:path, color=:purple, alpha=0.5, label="")
        end

        # --- Styling ---
        plot!(p, subplot=i,
            title = "N = $N",
            xlabel = "Index",
            # Only show Y-axis label for the leftmost plots (1 and 5)
            ylabel = (i % 4 == 1 ? "Coeffs (log)" : ""),
            yaxis = :log10,
            framestyle = :box
        )
    end

    return p
end

# Define a representative subset of N values to fit the 2x4 grid
grid_Ns = [10, 20, 30, 40, 50, 60, 70, 80]

# Execute and display
p_grid = plot_tail_comparison(results_00, results_002, N_subset=grid_Ns)
display(p_grid)

In [ ]:
"""
Plots both spectra on the same plot for a single system size N.
"""
function plot_single_N_comparison(results_1::Dict, results_2::Dict, N::Int; 
                                  label1="σ=0.0", label2="σ=0.002")
    
    # Error check
    if !haskey(results_1, N) && !haskey(results_2, N)
        error("N=$N found in neither results dictionary.")
    end

    p = plot(
        title = "Schmidt Spectrum Comparison (N=$N)",
        xlabel = "Schmidt Index",
        ylabel = "Coefficient Value",
        yaxis = :log10,
        framestyle = :box,
        grid = true,
        legend = :topright,
        size = (800, 600)
    )

    # --- Plot Dataset 2 (Orange) ---
    if haskey(results_2, N)
        v2 = sort(results_2[N], rev=true)
        mask2 = v2 .> 0
        y2 = v2[mask2]
        x2 = (1:length(v2))[mask2]

        plot!(p, x2, y2, 
            seriestype=:scatter, markershape=:circle, markersize=4, 
            markerstrokewidth=0, color=:darkorange, alpha=0.8, label=label2)
        plot!(p, x2, y2, seriestype=:path, color=:darkorange, alpha=0.5, label="")
    end

    # --- Plot Dataset 1 (Purple) ---
    if haskey(results_1, N)
        v1 = sort(results_1[N], rev=true)
        mask1 = v1 .> 0
        y1 = v1[mask1]
        x1 = (1:length(v1))[mask1]

        plot!(p, x1, y1, 
            seriestype=:scatter, markershape=:rect, markersize=4, 
            markerstrokewidth=0, color=:purple, alpha=0.5, label=label1)
        plot!(p, x1, y1, seriestype=:path, color=:purple, alpha=0.5, label="")
    end
    
    return p
end

# Execute for N=50 (or change to any N available in your data)
if haskey(results_002, 50)
    p_single = plot_single_N_comparison(results_00, results_002, 50)
    display(p_single)
else
    println("N=50 not available. Please try another N.")
end

In [ ]:
"""
Calculates the index where the clean (σ=0.0) and disordered (σ=0.002) Schmidt spectra deviate.
Returns a dictionary mapping N to the deviation index.

Criteria:
The deviation index is defined as the first index 'i' where the relative difference 
between the coefficients exceeds the `threshold`.
"""
function calculate_deviation_indices(results_clean::Dict, results_disordered::Dict; 
                                     threshold=0.1, N_subset=nothing)
    
    deviation_indices = Dict{Int, Int}()
    
    # Determine which N values to process
    available_Ns = intersect(keys(results_clean), keys(results_disordered))
    if N_subset !== nothing
        available_Ns = intersect(available_Ns, N_subset)
    end
    
    for N in sort(collect(available_Ns))
        # Get sorted coefficients (descending)
        v_clean = sort(results_clean[N], rev=true)
        v_disorder = sort(results_disordered[N], rev=true)
        
        # We can only compare up to the length of the shortest vector
        # (Usually the clean system has a smaller bond dimension/cutoff)
        min_len = min(length(v_clean), length(v_disorder))
        
        dev_idx = min_len # Default to the end if no deviation found
        
        for i in 1:min_len
            val_c = v_clean[i]
            val_d = v_disorder[i]
            
            # Avoid division by zero
            if val_c < 1e-16 || val_d < 1e-16
                # If one is zero and the other is not (significantly), that is the deviation
                if abs(val_c - val_d) > 1e-10
                    dev_idx = i
                    break
                end
                continue
            end
            
            # Relative difference metric: |c - d| / max(c, d)
            rel_diff = abs(val_c - val_d) / max(val_c, val_d)
            
            if rel_diff > threshold
                dev_idx = i
                break
            end
        end
        
        deviation_indices[N] = dev_idx
    end
    
    return deviation_indices
end


# You can adjust 'threshold' to define how sensitive the deviation detection is.
# 0.2 means a 20% relative difference triggers the deviation.
dev_indices = calculate_deviation_indices(results_00, results_002, threshold=0.2)

println("Deviation Indices calculated for N: ", sort(collect(keys(dev_indices))))

In [ ]:


"""
Calculates the 'weight' (sum of squared coefficients) of the tail and the head.
- Tail: Coefficients from `index` to the end.
- Head: Coefficients from 1 to `index - 1`.
"""
function calculate_weights(results_target::Dict, indices::Dict)
    tail_weights = Dict{Int, Float64}()
    weight_diffs = Dict{Int, Float64}() # Head - Tail
    
    for (N, idx) in indices
        if !haskey(results_target, N) continue end
        
        coeffs = sort(results_target[N], rev=true)
        
        # Calculate Tail Weight: Sum(λ_i^2) for i >= idx
        # Note: Schmidt coefficients are singular values, so we square them for probability weight.
        if idx <= length(coeffs)
            w_tail = sum(coeffs[idx:end] .^ 2)
        else
            w_tail = 0.0
        end
        
        # Calculate Head Weight: Sum(λ_i^2) for i < idx
        if idx > 1
            w_head = sum(coeffs[1:idx-1] .^ 2)
        else
            w_head = 0.0
        end
        
        tail_weights[N] = w_tail
        weight_diffs[N] = w_head - w_tail
    end
    
    return tail_weights, weight_diffs
end



# We calculate weights for the DISORDERED case (results_002) using the deviation index found.
tail_w, w_diffs = calculate_weights(results_002, dev_indices)

In [ ]:
"""
Plots the deviation indices and the weight analysis.
"""
function plot_analysis_results(deviation_indices::Dict, tail_weights::Dict, weight_diffs::Dict)
    
    # Sort N values for consistent plotting
    Ns = sort(collect(keys(deviation_indices)))
    
    # Prepare data arrays
    indices = [deviation_indices[N] for N in Ns]
    tails = [get(tail_weights, N, NaN) for N in Ns]
    diffs = [get(weight_diffs, N, NaN) for N in Ns]
    
    # --- Plot 1: Deviation Index vs N ---
    p1 = plot(Ns, indices,
        title = "Index of Deviation (σ=0 vs σ=0.002)",
        xlabel = "System Size (N)",
        ylabel = "Schmidt Index k",
        seriestype = :scatter,
        label = "Deviation Index",
        color = :blue,
        markershape = :diamond,
        markerstrokewidth = 0,
        markersize = 6,
        grid = true,
        framestyle = :box,
        legend = :topleft
    )
    plot!(p1, Ns, indices, seriestype=:line, color=:blue, alpha=0.5, label="")

    # --- Plot 2: Tail Weight vs N ---
    p2 = plot(Ns, tails,
        title = "Tail Weight (Σλ² after index k)",
        xlabel = "System Size (N)",
        ylabel = "Weight",
        seriestype = :scatter,
        label = "Tail Weight",
        color = :red,
        markershape = :circle,
        markersize = 5,
        markerstrokewidth = 0,
        grid = true,
        framestyle = :box,
        legend = :topleft
    )
    plot!(p2, Ns, tails, seriestype=:line, color=:red, alpha=0.5, label="")

    # --- Plot 3: Weight Difference (Head - Tail) vs N ---
    p3 = plot(Ns, diffs,
        title = "Weight Diff (Head - Tail)",
        xlabel = "System Size (N)",
        ylabel = "W_head - W_tail",
        seriestype = :scatter,
        label = "Difference",
        color = :green,
        markershape = :rect,
        markersize = 5,
        markerstrokewidth = 0,
        grid = true,
        framestyle = :box,
        legend = :bottomleft
    )
    plot!(p3, Ns, diffs, seriestype=:line, color=:green, alpha=0.5, label="")

    # Combine into a layout
    l = @layout [a{0.5h}; b c]
    p_final = plot(p1, p2, p3, layout=l, size=(1000, 800), margin=5Plots.mm)
    
    return p_final
end

p_analysis = plot_analysis_results(dev_indices, tail_w, w_diffs)
display(p_analysis)